In [56]:
import numpy as np
import pandas as pd
import calendar

In [57]:
df = pd.read_excel("../data/denormalized-data.xlsx")
df

,Row ID,Order ID,Order Date,Delivery Date,Delivery Mode,Customer ID,Customer Name,Customer Segment,City,Country,Region,Product ID,Product Category,Sub-Category,Product Name,Unit cost,Quantity,Selling price,Discount
0,1,OrdID-2018-0000011,2018-01-06,2018-01-13,5-7 Day,CustID- 401,Selorm Addo,Consumer,Tamale,Ghana,Northern,ProdID-28000011,Electronics,Accessories_Supplies,Power Supply Module for HKC 401-2K201-D4211 HK...,150,11,212,0.074905
1,2,OrdID-2018-0000021,2018-01-06,2018-01-08,2-3 Day,CustID- 525,Peter Ankoma,Consumer,Axim,Ghana,Western,ProdID-28000021,Phone_Tablets,Mobile Phones,Apple iPhone 8 Plus Gold 64GB 256GB 4G LTE Unl...,1754,5,2105,0.000000
2,3,OrdID-2018-0000031,2018-01-06,2018-01-08,2-3 Day,CustID- 214,Priscilla Mintah,Consumer,Tamale,Ghana,Northern,ProdID-28000031,Home_Office,Furniture,Printed Chair Cover Soft Milk Silk,290,8,343,0.050000
3,4,OrdID-2018-0000041,2018-01-06,2018-01-09,2-3 Day,CustID- 030,Cecilia Esi,Home Office,Ahwiaa,Ghana,Ashanti,ProdID-28000041,Phone_Tablets,Mobile Phones,Apple iphone 8 plus locked ee red - 256 gb,1418,6,1746,0.080000
4,5,OrdID-2018-0000051,2018-01-06,2018-01-08,2-3 Day,CustID- 204,Francis Mensah,Consumer,Tarkwa,Ghana,Western,ProdID-28000051,Electronics,Accessories_Supplies,Power Supply Board Driver Board for Samsung T2...,289,10,372,0.120000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,659,OrdID-2019-0006591,2019-12-22,2019-12-24,Pick up,CustID- 254,Krobo Edusei,Corporate,Tarkwa,Ghana,Western,ProdID-28000531,Phone_Tablets,Telephones_Accessories,"Telephone Cord, Phone Cord, Handset Cord, Blac...",553,8,780,0.039849
658,660,OrdID-2019-0006601,2019-12-26,2019-12-31,5-7 Day,CustID- 401,Selorm Addo,Consumer,Tamale,Ghana,Northern,ProdID-28000641,Phone_Tablets,Telephones_Accessories,iMBAPrice 50 Feet Long Telephone Extension Cor...,696,14,843,0.020000
659,661,OrdID-2019-0006611,2019-12-29,2019-12-29,Pick up,CustID- 210,Justice Nyamekye,Consumer,Bolgatanga,Ghana,Upper East,ProdID-28000171,Electronics,Accessories_Supplies,TV One 1RK-4RU-PSU 4RU 250w Power supply and a...,1121,14,1649,0.073850
660,662,OrdID-2019-0006621,2019-12-30,2019-12-31,Express 1 Day,CustID- 453,Osei Bonsu,Corporate,Tamale,Ghana,Northern,ProdID-28000691,Electronics,Home Audio,Pioneer VSX-451 AM/FM Pro-Logic Home Audio Ste...,1421,7,2104,0.125951


In [58]:
df["Revenue"] = df["Quantity"] * df["Selling price"]
df["Revenue"].sum()

6998937

In [59]:
df["Cost"] = df["Quantity"] * df["Unit cost"]
df["Cost"].sum()

5270143

In [60]:
df["Profit"] = df["Revenue"] - df["Cost"]
df["Profit"].sum()

1728794

In [61]:
df["Profit Margin"] = df["Profit"]/df["Revenue"]
df["Profit Margin"].mean()

0.24729130544498248

In [62]:
months = df["Order Date"].dt.month
try:
    months = months.apply(lambda x: calendar.month_abbr[x])
except(TypeError):
    months = months
# trend_df = df.groupby([months]).agg(
    # {"Revenue":"sum", "Cost":"sum", "Profit":'sum', "Profit Margin":'mean'})
order = calendar.month_abbr
trend_df = df.groupby([months])[["Revenue", "Cost", "Profit"]].sum()
trend_df["Profit Margin"] = trend_df["Profit"]/trend_df["Revenue"]
trend_df = trend_df.reindex(order, axis=0)
trend_df.dropna(inplace=True)
trend_df = trend_df.reset_index()


trend_df

,Order Date,Revenue,Cost,Profit,Profit Margin
0,Jan,506145.0,376940.0,129205.0,0.255273
1,Feb,306517.0,230723.0,75794.0,0.247275
2,Mar,767026.0,560592.0,206434.0,0.269136
3,Apr,796906.0,597874.0,199032.0,0.249756
4,May,570425.0,437312.0,133113.0,0.233358
5,Jun,298017.0,233627.0,64390.0,0.216061
6,Jul,682283.0,508509.0,173774.0,0.254695
7,Aug,496796.0,377420.0,119376.0,0.240292
8,Sep,668480.0,518723.0,149757.0,0.224026
9,Oct,682572.0,511566.0,171006.0,0.250532


In [63]:
regions_df = df.groupby('Region')["Profit"].sum().sort_values(ascending=False)
regions_df = regions_df.reset_index()
regions_df["Region"] = regions_df["Region"].str.replace("Brong-Ahafo", "Brong Ahafo")
regions_df


,Region,Profit
0,Northern,400787
1,Ashanti,245647
2,Greater Accra,208897
3,Volta,184245
4,Western,183871
5,Central,112227
6,Upper West,111352
7,Upper East,104895
8,Eastern,91101
9,Brong Ahafo,85772


In [64]:
sub_category_df = df.groupby('Sub-Category')[["Revenue", "Cost", "Profit"]].sum().sort_values(by="Revenue",ascending=False)
sub_category_df["PM Tracker"] = sub_category_df["Profit"]/sub_category_df["Revenue"]
sub_category_df.drop("Profit",axis=1, inplace=True)
sub_category_df.loc['Grand Total'] = sub_category_df[["Revenue", "Cost"]].sum()
sub_category_df["PM Tracker"].loc['Grand Total'] = sub_category_df["PM Tracker"].mean()
sub_category_df = sub_category_df.reset_index()

sub_category_df

,Sub-Category,Revenue,Cost,PM Tracker
0,Television,2001334.0,1539031.0,0.230997
1,Mobile Phones,1224368.0,913582.0,0.253834
2,Telephones_Accessories,894663.0,679347.0,0.240667
3,Home_Kitchen,887160.0,662944.0,0.252735
4,Accessories_Supplies,471821.0,340528.0,0.278269
5,Laptop_Desktop accessories,441140.0,328087.0,0.256275
6,Wearable Technology,252970.0,191611.0,0.242554
7,Home Audio,218385.0,159715.0,0.268654
8,Furniture,182888.0,136376.0,0.254320
9,Headphones,178267.0,131811.0,0.260598


In [65]:
sub_category_df["Revenue"] = sub_category_df["Revenue"].apply(lambda x: "{:,.1f}k".format((x/1000)))
sub_category_df["Cost"] = sub_category_df["Cost"].apply(lambda x: "{:,.1f}k".format((x/1000)))
sub_category_df

,Sub-Category,Revenue,Cost,PM Tracker
0,Television,"2,001.3k","1,539.0k",0.230997
1,Mobile Phones,"1,224.4k",913.6k,0.253834
2,Telephones_Accessories,894.7k,679.3k,0.240667
3,Home_Kitchen,887.2k,662.9k,0.252735
4,Accessories_Supplies,471.8k,340.5k,0.278269
5,Laptop_Desktop accessories,441.1k,328.1k,0.256275
6,Wearable Technology,253.0k,191.6k,0.242554
7,Home Audio,218.4k,159.7k,0.268654
8,Furniture,182.9k,136.4k,0.254320
9,Headphones,178.3k,131.8k,0.260598


In [66]:
sub_category_df["PM Tracker"] = sub_category_df["PM Tracker"].apply(lambda x: '🔴' if x<0.2
else '🟡' if x<0.25
else '🟢')

sub_category_df

,Sub-Category,Revenue,Cost,PM Tracker
0,Television,"2,001.3k","1,539.0k",🟡
1,Mobile Phones,"1,224.4k",913.6k,🟢
2,Telephones_Accessories,894.7k,679.3k,🟡
3,Home_Kitchen,887.2k,662.9k,🟢
4,Accessories_Supplies,471.8k,340.5k,🟢
5,Laptop_Desktop accessories,441.1k,328.1k,🟢
6,Wearable Technology,253.0k,191.6k,🟡
7,Home Audio,218.4k,159.7k,🟢
8,Furniture,182.9k,136.4k,🟢
9,Headphones,178.3k,131.8k,🟢


In [67]:
# def red_circle(v, props=""):
#     return props if v <0.2 else None


# sub_category_df[["PM Tracker"]] = sub_category_df[["PM Tracker"]].style.applymap(red_circle,
#  props="height: 25px; width: 25px; background-color: #c85533; border-radius: 50%;")


# sub_category_df

,Sub-Category,Revenue,Cost,PM Tracker
0,Television,"2,001.3k","1,539.0k",<pandas.io.formats.style.Styler object at 0x00...
1,Mobile Phones,"1,224.4k",913.6k,<pandas.io.formats.style.Styler object at 0x00...
2,Telephones_Accessories,894.7k,679.3k,<pandas.io.formats.style.Styler object at 0x00...
3,Home_Kitchen,887.2k,662.9k,<pandas.io.formats.style.Styler object at 0x00...
4,Accessories_Supplies,471.8k,340.5k,<pandas.io.formats.style.Styler object at 0x00...
5,Laptop_Desktop accessories,441.1k,328.1k,<pandas.io.formats.style.Styler object at 0x00...
6,Wearable Technology,253.0k,191.6k,<pandas.io.formats.style.Styler object at 0x00...
7,Home Audio,218.4k,159.7k,<pandas.io.formats.style.Styler object at 0x00...
8,Furniture,182.9k,136.4k,<pandas.io.formats.style.Styler object at 0x00...
9,Headphones,178.3k,131.8k,<pandas.io.formats.style.Styler object at 0x00...


In [68]:

ghana_geo = pd.read_json("../data/ghana_regions.json")
ghana_geo = ghana_geo["features"]
ghana_geo

0    {'type': 'Feature', 'geometry': {'type': 'Poly...
1    {'type': 'Feature', 'geometry': {'type': 'Poly...
2    {'type': 'Feature', 'geometry': {'type': 'Mult...
3    {'type': 'Feature', 'geometry': {'type': 'Poly...
4    {'type': 'Feature', 'geometry': {'type': 'Mult...
5    {'type': 'Feature', 'geometry': {'type': 'Poly...
6    {'type': 'Feature', 'geometry': {'type': 'Poly...
7    {'type': 'Feature', 'geometry': {'type': 'Poly...
8    {'type': 'Feature', 'geometry': {'type': 'Mult...
9    {'type': 'Feature', 'geometry': {'type': 'Mult...
Name: features, dtype: object

In [69]:
import plotly.express as px
fig = px.choropleth(regions_df, geojson=ghana_geo, scope="africa", labels="Profit",locations="Region")
#fig.show()

In [70]:
#ghana_geo[0]

In [71]:
regions_geo = []
for region in ghana_geo:
    region_name = region["properties"]['NAME_1']
    geometry = region["geometry"]
    regions_geo.append({"type":"Feature", 
                        "geometry":geometry,
                        "id":region_name})
#regions_geo[0]

In [72]:
regions_geo_full = {"type":"Feature Collection", "features":regions_geo}

In [73]:
# fig = px.choropleth(regions_df, 
# geojson = regions_geo_full,
# color = regions_df["Profit"])

# fig.update_layout(
#     margin={'r':0,'t':0,'l':0,'b':0},
#     coloraxis_colorbar={
#         'title':'Profit per region',
#         'tickvals':regions_df["Profit"]     
#     }
# )
# fig.show()

In [74]:
def create_date_table(start='2018-01-01', end='2019-12-31'):
    date_df = pd.DataFrame({"Date": pd.date_range(start, end)})
    date_df["Day"] = date_df.Date.dt.day
    date_df["Month"] = date_df.Date.dt.month
    date_df["Month"] = date_df["Month"].apply(lambda x: calendar.month_abbr[x])
    date_df["Quarter"] = date_df.Date.dt.quarter
    date_df["Year"] = date_df.Date.dt.year
    return date_df

create_date_table()

,Date,Day,Month,Quarter,Year
0,2018-01-01,1,Jan,1,2018
1,2018-01-02,2,Jan,1,2018
2,2018-01-03,3,Jan,1,2018
3,2018-01-04,4,Jan,1,2018
4,2018-01-05,5,Jan,1,2018
...,...,...,...,...,...
725,2019-12-27,27,Dec,4,2019
726,2019-12-28,28,Dec,4,2019
727,2019-12-29,29,Dec,4,2019
728,2019-12-30,30,Dec,4,2019


In [75]:
date_df = create_date_table()
date_df = date_df[date_df["Year"].isin([2019])]
date_df = date_df["Day"]
type(int(np.asarray(date_df.min())))

int

In [76]:
"".join(["Months"])

'Months'

In [77]:
df["Years"] = df["Delivery Date"].dt.year
df["Months"] = df["Delivery Date"].dt.month
df["Quarters"] = df["Delivery Date"].dt.quarter
df["Days"] = df["Delivery Date"].dt.day

period = "".join(["Months"])
df = df[df["Years"].isin([2018, 2019])]
df = df[df[period].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])]

profit = df["Profit"]
profit = "{:,}".format(profit.sum())
profit

'1,728,794'

In [78]:
dict_ = df.to_dict("series")
dict_

{'Row ID': 0        1
 1        2
 2        3
 3        4
 4        5
       ... 
 657    659
 658    660
 659    661
 660    662
 661    663
 Name: Row ID, Length: 662, dtype: int64,
 'Order ID': 0      OrdID-2018-0000011
 1      OrdID-2018-0000021
 2      OrdID-2018-0000031
 3      OrdID-2018-0000041
 4      OrdID-2018-0000051
               ...        
 657    OrdID-2019-0006591
 658    OrdID-2019-0006601
 659    OrdID-2019-0006611
 660    OrdID-2019-0006621
 661    OrdID-2019-0006631
 Name: Order ID, Length: 662, dtype: object,
 'Order Date': 0     2018-01-06
 1     2018-01-06
 2     2018-01-06
 3     2018-01-06
 4     2018-01-06
          ...    
 657   2019-12-22
 658   2019-12-26
 659   2019-12-29
 660   2019-12-30
 661   2019-12-30
 Name: Order Date, Length: 662, dtype: datetime64[ns],
 'Delivery Date': 0     2018-01-13
 1     2018-01-08
 2     2018-01-08
 3     2018-01-09
 4     2018-01-08
          ...    
 657   2019-12-24
 658   2019-12-31
 659   2019-12-29
 660   2019-12-3

In [79]:
df1 = pd.DataFrame(dict_)
df1

,Row ID,Order ID,Order Date,Delivery Date,Delivery Mode,Customer ID,Customer Name,Customer Segment,City,Country,...,Selling price,Discount,Revenue,Cost,Profit,Profit Margin,Years,Months,Quarters,Days
0,1,OrdID-2018-0000011,2018-01-06,2018-01-13,5-7 Day,CustID- 401,Selorm Addo,Consumer,Tamale,Ghana,...,212,0.074905,2332,1650,682,0.292453,2018,1,1,13
1,2,OrdID-2018-0000021,2018-01-06,2018-01-08,2-3 Day,CustID- 525,Peter Ankoma,Consumer,Axim,Ghana,...,2105,0.000000,10525,8770,1755,0.166746,2018,1,1,8
2,3,OrdID-2018-0000031,2018-01-06,2018-01-08,2-3 Day,CustID- 214,Priscilla Mintah,Consumer,Tamale,Ghana,...,343,0.050000,2744,2320,424,0.154519,2018,1,1,8
3,4,OrdID-2018-0000041,2018-01-06,2018-01-09,2-3 Day,CustID- 030,Cecilia Esi,Home Office,Ahwiaa,Ghana,...,1746,0.080000,10476,8508,1968,0.187858,2018,1,1,9
4,5,OrdID-2018-0000051,2018-01-06,2018-01-08,2-3 Day,CustID- 204,Francis Mensah,Consumer,Tarkwa,Ghana,...,372,0.120000,3720,2890,830,0.223118,2018,1,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,659,OrdID-2019-0006591,2019-12-22,2019-12-24,Pick up,CustID- 254,Krobo Edusei,Corporate,Tarkwa,Ghana,...,780,0.039849,6240,4424,1816,0.291026,2019,12,4,24
658,660,OrdID-2019-0006601,2019-12-26,2019-12-31,5-7 Day,CustID- 401,Selorm Addo,Consumer,Tamale,Ghana,...,843,0.020000,11802,9744,2058,0.174377,2019,12,4,31
659,661,OrdID-2019-0006611,2019-12-29,2019-12-29,Pick up,CustID- 210,Justice Nyamekye,Consumer,Bolgatanga,Ghana,...,1649,0.073850,23086,15694,7392,0.320194,2019,12,4,29
660,662,OrdID-2019-0006621,2019-12-30,2019-12-31,Express 1 Day,CustID- 453,Osei Bonsu,Corporate,Tamale,Ghana,...,2104,0.125951,14728,9947,4781,0.324620,2019,12,4,31
